<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [4]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [5]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [28]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

OperationalError: table houses already exists

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [34]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command

c.execute(sql_command, values)

# Remember to commit the changes
conn.commit()

NameError: name 'sql_command' is not defined

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [39]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:

c.execute('INSERT INTO houses VALUES (?,?,?,?,?);', last_sale)

conn.commit()

Select all rows from houses

In [43]:
# A
c.execute('SELECT * FROM houses;')
conn.commit()
c.fetchall()

[(1, 4000, 5, 22, 619000), (2, 4000, 5, 22, 619000)]

In [45]:
#alternative
for row in c.execute('SELECT * FROM houses'):
        print(row)

(1, 4000, 5, 22, 619000)
(2, 4000, 5, 22, 619000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [54]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../../../DATA/housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [47]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [48]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [49]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [50]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [51]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [72]:
# ANSWER
results = c.execute('SELECT * FROM houses WHERE bdrms=4;')
conn.commit()
results.fetchall()

[(7, 3000, 4, 75, 539900),
 (8, 1985, 4, 61, 299900),
 (13, 1940, 4, 7, 239999),
 (18, 2300, 4, 77, 449900),
 (21, 2609, 4, 5, 499998),
 (22, 3031, 4, 21, 599000),
 (26, 1962, 4, 53, 259900),
 (35, 2040, 4, 75, 314900),
 (37, 1811, 4, 24, 285900),
 (40, 2132, 4, 28, 345000),
 (41, 4215, 4, 66, 549000),
 (42, 2162, 4, 43, 287000),
 (45, 2567, 4, 57, 314000),
 (48, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [73]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1814.1923076923076, 323465.3076923077),
 (4, 2400.4285714285716, 377449.78571428574),
 (5, 4159.333333333333, 645966.6666666666)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [74]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [75]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../../../DATA/housing-data.csv', low_memory = False)
data.head()

sqft  bdrms  age   price
0  2104      3   70  399900
1  1600      3   28  329900
2  2400      3   44  369000
3  1416      2   49  232000
4  3000      4   75  539900

We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [76]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [79]:
#ANSWER
pd.read_sql_query('SELECT AVG(price) AS avg_price FROM houses_pandas GROUP BY sqft;',conn)

avg_price
0    179900.0
1    169900.0
2    249900.0
3    299000.0
4    239500.0
5    199900.0
6    229900.0
7    259900.0
8    299900.0
9    212000.0
10   232000.0
11   198999.0
12   249900.0
13   464500.0
14   242500.0
15   314900.0
16   329900.0
17   242900.0
18   368500.0
19   252900.0
20   285900.0
21   349900.0
22   299900.0
23   255000.0
24   329999.0
25   239999.0
26   259900.0
27   299900.0
28   347000.0
29   314900.0
30   399900.0
31   345000.0
32   287000.0
33   475000.0
34   329900.0
35   449900.0
36   369000.0
37   469000.0
38   314000.0
39   499998.0
40   299900.0
41   539900.0
42   599000.0
43   579900.0
44   573900.0
45   549000.0
46   699900.0

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [111]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [112]:
# Reading CSV to Dataframe
orders = pd.read_csv('../../../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('../../../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [113]:
orders.head()

Order ID  Order Date      Customer Name         Country
0  IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1  ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2  ES-2011-4939443  2011-01-04     Arthur Prichep          France
3  IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4  ES-2011-3848439  2011-01-05   Michael Granlund          France

In [114]:
orders_break_down.head()

Order ID                           Product Name  Discount  Sales  \
0  IT-2011-3647632            Enermax Note Cards, Premium       0.5   45.0   
1  ES-2011-4869686     Dania Corner Shelving, Traditional       0.0  854.0   
2  ES-2011-4939443  Binney & Smith Sketch Pad, Easy-Erase       0.0  140.0   
3  IT-2011-2942451             Boston Markers, Easy-Erase       0.5   27.0   
4  IT-2011-2942451            Eldon Folders, Single Width       0.5   17.0   

   Quantity         Category  
0         3  Office Supplies  
1         7        Furniture  
2         3  Office Supplies  
3         2  Office Supplies  
4         2  Office Supplies

#### 2. Replace Space with Underscore in Column Names

In [115]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [116]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [117]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [118]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [119]:
# Establishing Local DB connection
db_connection = sqlite3.connect('../../../DATA/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders_sq', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down_sq', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [139]:
# Select first 10 rows of all columns in orders table

orders_head_10 = sql.read_sql('SELECT * FROM orders_sq LIMIT 10;', con=db_connection)
orders_head_10

order_id  order_date      customer_name         country
0  IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1  ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2  ES-2011-4939443  2011-01-04     Arthur Prichep          France
3  IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4  ES-2011-3848439  2011-01-05   Michael Granlund          France
5  ES-2011-5433855  2011-01-07       Dave Poirier          France
6  IT-2011-4546695  2011-01-08      Darren Powers          France
7  ES-2011-1138719  2011-01-11       Eric Murdock           Italy
8  ES-2011-1466305  2011-01-11         Mick Brown         Austria
9  ES-2011-4359424  2011-01-11  Dorothy Dickinson           Spain

In [158]:
# Select Number of Orders for Each Customer
sql.read_sql('SELECT COUNT(order_id) AS order_count FROM orders_sq GROUP BY customer_name;',con=db_connection)

order_count
0             11
1              7
2             12
3              4
4              8
..           ...
787            2
788            5
789           11
790            9
791            6

[792 rows x 1 columns]

#### 6. Select Number of Customers for Each Country

In [159]:
# Select Number of Customers for Each Country
sql.read_sql('SELECT COUNT(customer_name) AS customer_count FROM orders_sq GROUP BY country;',con=db_connection)

customer_count
0              135
1               68
2               29
3               34
4              991
5              806
6               50
7              493
8              194
9               37
10              37
11             403
12             100
13              40
14             700

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [160]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
sql.read_sql('SELECT discount, sales, quantity FROM orders_break_down_sq GROUP BY order_id;',con=db_connection)

discount   sales  quantity
0          0.1   229.0         9
1          0.0    58.0         2
2          0.0   148.0         3
3          0.0   224.0        11
4          0.0    13.0         2
...        ...     ...       ...
4112       0.5   107.0         4
4113       0.1   656.0         2
4114       0.5    83.0         1
4115       0.1  1603.0         9
4116       0.0    74.0         8

[4117 rows x 3 columns]

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [161]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
sql.read_sql('SELECT discount, sales, quantity, (sales*quantity-discount) AS total_price FROM orders_break_down_sq GROUP BY order_id;',con=db_connection)

discount   sales  quantity  total_price
0          0.1   229.0         9       2060.9
1          0.0    58.0         2        116.0
2          0.0   148.0         3        444.0
3          0.0   224.0        11       2464.0
4          0.0    13.0         2         26.0
...        ...     ...       ...          ...
4112       0.5   107.0         4        427.5
4113       0.1   656.0         2       1311.9
4114       0.5    83.0         1         82.5
4115       0.1  1603.0         9      14426.9
4116       0.0    74.0         8        592.0

[4117 rows x 4 columns]

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [157]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
sql.read_sql('SELECT * FROM orders_break_down_sq WHERE sales*quantity-discount > 100;',con=db_connection)

order_id                                  product_name  discount  \
0     IT-2011-3647632                   Enermax Note Cards, Premium       0.5   
1     ES-2011-4869686            Dania Corner Shelving, Traditional       0.0   
2     ES-2011-4939443         Binney & Smith Sketch Pad, Easy-Erase       0.0   
3     ES-2011-3848439  Binney & Smith Pencil Sharpener, Water Color       0.0   
4     ES-2011-3848439                   Sanford Canvas, Fluorescent       0.0   
...               ...                                           ...       ...   
6461  ES-2014-2815584                     Kleencut Shears, Serrated       0.0   
6462  ES-2014-4785777               Wilson Jones Index Tab, Economy       0.0   
6463  ES-2014-3458802          Ikea Stackable Bookrack, Traditional       0.0   
6464  IT-2014-3715679                           StarTech Phone, Red       0.5   
6465  ES-2014-3458802                     Motorola Audio Dock, VoIP       0.0   

      sales  quantity         category  
0      45.0         3  Office Supplies  
1     854.0         7        Furniture  
2     140.0         3  Office Supplies  
3      90.0         3  Office Supplies  
4     207.0         4  Office Supplies  
...     ...       ...              ...  
6461  261.0         6  Office Supplies  
6462   32.0         5  Office Supplies  
6463  245.0         2        Furniture  
6464  108.0         3       Technology  
6465  867.0         5       Technology  

[6466 rows x 6 columns]

#### 8. Select All Customer And The Product They Have Bought

In [165]:
# Select All Customer And The Product They Have Bought
# using WHERE statement
sql.read_sql('SELECT customer_name, product_name FROM orders_sq, orders_break_down_sq WHERE orders_sq.order_id=orders_break_down_sq.order_id GROUP BY customer_name;',con=db_connection)

customer_name                               product_name
0         Aaron Bergman           Apple Office Telephone, Cordless
1         Aaron Hawkins                     Epson Inkjet, Wireless
2        Aaron Smayling           Binney & Smith Pens, Fluorescent
3       Adam Bellavance            Sauder Classic Bookcase, Mobile
4             Adam Hart            Cardinal Binder Covers, Economy
..                  ...                                        ...
787        Xylona Preis              Ikea Classic Bookcase, Mobile
788       Yana Sorensen     Binney & Smith Sketch Pad, Water Color
789      Yoseph Carroll           Hon Round Labels, 5000 Label Set
790    Zuschuss Carroll                           BIC Canvas, Blue
791  Zuschuss Donatelli  Barricks Round Table, with Bottom Storage

[792 rows x 2 columns]

In [166]:
# using JOIN statement
sql.read_sql('SELECT customer_name, product_name FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id GROUP BY customer_name;',con=db_connection)

customer_name                               product_name
0         Aaron Bergman           Apple Office Telephone, Cordless
1         Aaron Hawkins                     Epson Inkjet, Wireless
2        Aaron Smayling           Binney & Smith Pens, Fluorescent
3       Adam Bellavance            Sauder Classic Bookcase, Mobile
4             Adam Hart            Cardinal Binder Covers, Economy
..                  ...                                        ...
787        Xylona Preis              Ikea Classic Bookcase, Mobile
788       Yana Sorensen     Binney & Smith Sketch Pad, Water Color
789      Yoseph Carroll           Hon Round Labels, 5000 Label Set
790    Zuschuss Carroll                           BIC Canvas, Blue
791  Zuschuss Donatelli  Barricks Round Table, with Bottom Storage

[792 rows x 2 columns]

#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [189]:
# Select Number of 'Furniture' Orders For Each Country
# using JOIN statement
sql.read_sql('SELECT country, COUNT(category) AS furniture_orders FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id WHERE category = "Furniture" GROUP BY country;',con=db_connection)

country  furniture_orders
0          Austria                40
1          Belgium                20
2          Denmark                 9
3          Finland                16
4           France               299
5          Germany               264
6          Ireland                12
7            Italy               136
8      Netherlands                76
9           Norway                 6
10        Portugal                10
11           Spain               123
12          Sweden                36
13     Switzerland                11
14  United Kingdom               180

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [191]:
# Select Number of 'Furniture' Orders For The Country Denmark
sql.read_sql('SELECT country, COUNT(category) AS furniture_orders FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id WHERE category = "Furniture" AND country = "Denmark";',con=db_connection)

country  furniture_orders
0  Denmark                 9

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [195]:
# Select Total Sales With Discount and Without Discount for Each Country
# CANT SEPERATE DISCOUNT??
sql.read_sql('SELECT country, SUM(sales) AS total_sales, discount FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id GROUP BY country;',con=db_connection)

country  total_sales  discount
0          Austria      79382.0       0.0
1          Belgium      42320.0       0.0
2          Denmark       7763.0       0.5
3          Finland      20702.0       0.0
4           France     609683.0       0.0
5          Germany     488681.0       0.0
6          Ireland      15998.0       0.5
7            Italy     252742.0       0.0
8      Netherlands      70313.0       0.5
9           Norway      20529.0       0.0
10        Portugal      15106.0       0.5
11           Spain     249402.0       0.0
12          Sweden      30490.0       0.5
13     Switzerland      24874.0       0.0
14  United Kingdom     420497.0       0.0

#### 11.A Select Total Quantity, Total Sales for Each Country

In [194]:
# Select Total Quantity, Total Sales for Each Country
sql.read_sql('SELECT country, SUM(quantity) AS total_quantity, SUM(sales) AS total_sales FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id GROUP BY country;',con=db_connection)

country  total_quantity  total_sales
0          Austria             973      79382.0
1          Belgium             532      42320.0
2          Denmark             204       7763.0
3          Finland             201      20702.0
4           France            7329     609683.0
5          Germany            6179     488681.0
6          Ireland             392      15998.0
7            Italy            3612     252742.0
8      Netherlands            1526      70313.0
9           Norway             261      20529.0
10        Portugal             286      15106.0
11           Spain            2881     249402.0
12          Sweden             753      30490.0
13     Switzerland             308      24874.0
14  United Kingdom            4917     420497.0

#### 11.B Select Top 3 Country Based on Sales

In [200]:
# Select Top 3 Country Based on Sales
# Select Total Quantity, Total Sales for Each Country
sql.read_sql('SELECT country, SUM(sales) AS total_sales FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id GROUP BY country ORDER BY total_sales DESC LIMIT 3;',con=db_connection)

country  total_sales
0          France     609683.0
1         Germany     488681.0
2  United Kingdom     420497.0

#### 11.C Select Bottom 3 Country Based On Quantities

In [202]:
# Select Bottom 3 Country Based On Quantities
sql.read_sql('SELECT country, SUM(quantity) AS total_quantity FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id GROUP BY country ORDER BY total_quantity LIMIT 3;',con=db_connection)

country  total_quantity
0  Finland             201
1  Denmark             204
2   Norway             261

#### 12. Select Average Sales By Categroy For The Country 'France'

In [203]:
# Select Average Sales By Categroy For The Country 'France'
sql.read_sql('SELECT category, country, AVG(sales) AS avg_sales FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id WHERE country = "France" GROUP BY category;',con=db_connection)

category country   avg_sales
0        Furniture  France  582.314381
1  Office Supplies  France  167.217709
2       Technology  France  595.145078

#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [208]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
sql.read_sql('SELECT country, category, SUM(sales) AS total_sales, AVG(sales) AS avg_sales FROM orders_sq JOIN orders_break_down_sq ON orders_sq.order_id=orders_break_down_sq.order_id GROUP BY country ORDER BY avg_sales DESC;',con=db_connection)

country         category  total_sales   avg_sales
0            Spain  Office Supplies     249402.0  327.729304
1          Finland  Office Supplies      20702.0  323.468750
2   United Kingdom        Furniture     420497.0  320.012938
3      Switzerland  Office Supplies      24874.0  318.897436
4           France  Office Supplies     609683.0  318.206159
5          Belgium  Office Supplies      42320.0  313.481481
6          Austria  Office Supplies      79382.0  300.689394
7          Germany  Office Supplies     488681.0  297.976220
8           Norway  Office Supplies      20529.0  293.271429
9            Italy  Office Supplies     252742.0  258.163432
10        Portugal       Technology      15106.0  215.800000
11     Netherlands  Office Supplies      70313.0  178.913486
12         Ireland       Technology      15998.0  159.980000
13          Sweden  Office Supplies      30490.0  150.197044
14         Denmark  Office Supplies       7763.0  129.383333

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



